# PREPROCESSING: set environment
set python environment

In [ ]:
do_installs = True
if do_installs:
    !pip install acoustics
    !pip install numpy
    !pip install scipy
    !pip install matplotlib
    !pip install mne
    !pip install pandas
    !pip install tensorpac
    !pip install requests


In [ ]:
import acoustics      #=> for creating artificial signals
import numpy as np    #=> for numeric operations
import scipy as sc    #=> for scientific computing
import matplotlib.pyplot as plt #=> for plotting
import mne            #=> for signal processing (M/EEG)

import copy           #=> deepcopying

import pandas as pd   #=> for working with data frames

%matplotlib inline    
                      #=> set plotting backend for notebooks

# PREPROCESSING: basic principles

## Preprocessing: WHY?
Raw electroencephalography data (recordings from depth electrodes: macro- and microelectrodes) contain artifacts that must be removed before the signal can be used for further analyses.
 

Potential artifacts:
* Electromagnetic noise
* Line noise
* Movement artifacts
* Signal jumps
* Potential drifts
* ...

## Preprocessing: HOW?
* Identify artifacts:
    * by visual inspection of raw data
    * by means of simple descriptive statistics (e.g. standard deviation of signal, ...)
* Remove artifacs:
  * Filtering
  * Re-referencing
  * Rejecting 
  * ...

## Artifact identification

### Visual inspection
* Look at the raw data
* Look at the power spectral density (PSD)

Create artificial signal:
* pink-noise data to mimick backround neuronal activity
* 10 s of data
* 1000 Hz sampling rate

In [ ]:
#%% Create artificial signal
ieeg = {}
ieeg['dur']   = 10
ieeg['srate'] = 1000
ieeg['time']  = np.arange(0,ieeg['dur']*ieeg['srate'])/ieeg['srate']

ieeg['data'] = acoustics.generator.noise(
    N=ieeg['dur']*ieeg['srate'],
    color='pink',  # pink noise: 1/f distribution
    state=np.random.RandomState(seed=2022) # seed for random numbers generator
    )



#### Inspection of raw signal
Plot artificial signal:

In [ ]:
#%% Plot artificial signal
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(ieeg['time'],ieeg['data'])

ax.set_title('Raw signal')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()

#### Inspection of power spectral density (PSD)
* A time-series can be described as a sum of oscillations of different frequencies that all have different amplitudes
* Inspecting the different frequency contents (the amplitudes oscillations at different frequencies have) can provide information about potential artifacts in the signal

Compute and plot the PSD of our signal:

In [ ]:
#%% Compute power spectral density (PSD) of signal
ieeg_psd_analytic,ieeg_psd_freqs = mne.time_frequency.psd_array_multitaper(
    x = ieeg['data'],
    sfreq = ieeg['srate'])

#%% Plot PSD of signal
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(ieeg_psd_freqs,ieeg_psd_analytic)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Amplitude')
ax.set_yscale("log")
ax.set_title('Power spectral denity')
ax.grid()

#### Explore data with different spectral properties

Function to create and plot different types of signals:

In [ ]:
def create_and_plot_artificial_signal(dur=10,srate = 1000, kind='pink'):
    """
    

    Parameters
    ----------
    dur : float, optional
        Duration of signal in seconds. The default is 10.
    srate : float, optional
        sampling-rate of signal. The default is 1000.
    kind : string({white, pink, blue, brown, violet}), optional
        Type of random noise that is generated. The default is 'pink'.

    Returns
    -------
    noise data: dict with fields
    data : np array containing the actual data
    srate : sampling rate 
    time : np array containing the time-stamps (in s) for each sample

    """
    # => some pink noise
    noise_dat = {}

    noise_dat['data'] = acoustics.generator.noise(
        N=dur*srate,        # 10`000 samples
        color=kind,  # white noise: all frequencyies are equally present
        state=np.random.RandomState(seed=2022) # seed for random numbers generator
        )

    noise_dat['srate'] = srate
    noise_dat['time'] = np.arange(0,len(noise_dat['data']))/noise_dat['srate']
    
    

    # Compute psd    
    noise_dat_psd_analytic,noise_dat_psd_freqs = mne.time_frequency.psd_array_multitaper(
        x = noise_dat['data'],
        sfreq = noise_dat['srate'])
        
    # Plot raw signal and PSD
    fig, ax = plt.subplots(nrows = 1,ncols = 2, figsize = (15,5))
    
    ax[0].plot(noise_dat['time'],noise_dat['data'])
    ax[0].set_title('Raw signal')
    ax[0].set_xlabel('Time (s)')
    ax[0].set_ylabel('Amplitude')
    ax[0].grid()
    
    
    ax[1].plot(noise_dat_psd_freqs,noise_dat_psd_analytic)
    ax[1].set_xlabel('Frequency (Hz)')
    ax[1].set_ylabel('Amplitude')
    ax[1].set_yscale("log")
    ax[1].set_title('Power spectral denity')
    ax[1].grid()
    fig.tight_layout()
    
    return noise_dat   

Explore different types of signals:
* white
* pink
* blue
* brown
* violet

In [ ]:
create_and_plot_artificial_signal(dur=10,srate = 1000, kind='pink')

ASSIGNMENT: Explore which type of signal looks like actual electrophysiological data from within the brain

In [ ]:
# ASSIGNMENT:
# => hint: use the above code, but plot different kinds of signal (white noise, ...)

#### EXCURSUS: time-domain vs. frequency domain
* Raw data are represented in the time-domain (voltage fluctuations across time)
* These fluctuations can be represented in the frequency domain: sum of oscillations (sine/cosine waves) that all have different ...
  * ... frequencies
  * ... amplitudes ("power")
  * ... phases

Function for creating a sum of oscillations:

In [ ]:
# Function for creating and plotting 
def sum_of_cosine_waves(duration, fsample, cosine_wave_params, do_plot = True):
    """
    Create signal by combining cosine-waves with different frequencies, 
    amplitudes, and phase-offsets, plot the signal

    Parameters
    ----------
    duration : float
        duration of artificial signal in seconds
    fsample : TYPE
        sampling rate of artificial signal
    cosine_wave_params : list of dicts
        list of dicts with amplitude, frequency, and phase-shift of signal.
        Fields:
            - amp: float
            - freq: float, in Hz 
            - phase: float, -PI to PI
    doplot: bool (default: True)
        whether or not to plot the data
        

    Returns
    -------
    data: dict
        Dictionnary containing the artificial signal. Keys:
            'time': time vector
            'data': the time series
            'srate': sampling rate
    fig: figure handle, [] if do_plot = False
    """
    dat = {}
    
    dat['srate'] = fsample
    dat['time'] = np.arange(0,duration*fsample)/fsample
    dat['type'] = ['eeg']
    dat['data'] = [np.zeros(dat['time'].shape)]
    
    
    for Wi in cosine_wave_params:
        dat['data'][0] = dat['data'][0] + \
            Wi['amp']*np.cos(dat['time']*2*np.pi*Wi['freq']+Wi['phase'])
        
    
        
    psd = mne.time_frequency.psd_array_multitaper(
        x = dat['data'][0],
        sfreq = dat['srate'],
        output = 'complex')
    
    dat['psd_pow']     = np.abs(np.mean(psd[0],0))
    dat['psd_phase']   = np.angle(np.mean(psd[0],0))
    dat['psd_freqs']   = psd[1]
    
    fig = []
    
    if do_plot:
        fig, ax = plt.subplots(nrows = 3,figsize = (15,10)) 
        
        ax[0].plot(dat['time'],dat['data'][0])
        ax[0].set_xlabel('Time (s)')
        ax[0].set_ylabel('Amplitude')
        ax[0].set_title('Raw signal')
        ax[0].grid()
        
        
        ax[1].plot(dat['psd_freqs'],dat['psd_pow'])
        ax[1].set_xlabel('Frequency (Hz)')
        ax[1].set_ylabel('Amplitude')
        ax[1].set_yscale('log')
        ax[1].set_title('Power spectral density')
        ax[1].grid()
        
        
        ax[2].plot(dat['psd_freqs'],dat['psd_phase'])
        ax[2].set_xlabel('Frequency (Hz)')
        ax[2].set_ylabel('Phase (Rad)')
        ax[2].set_ylim(-3.2,3.2)
        ax[2].set_title('Phase')
        ax[2].grid()
        
        fig.tight_layout()
            
    
    
    return dat, fig


Create and plot sum of oscillations: 1 Hz plus 20 Hz:

In [ ]:
ieeg, fig = sum_of_cosine_waves(10,500,
                        [{'amp':1,'freq':1,'phase':0},
                        {'amp':1,'freq':20,'phase':0}])

ASSIGNMENT: 
* Explore what happens when oscillations with very similar frequencies are combined (e.g. 10 and 11 Hz).
* Explore what happens when oscillations with frequencies that are an integer multiple of the fundamental frequency are combined.
* Explore what happens when oscillations are added that have frequencies close to the sampling frequency.
* Explore what happens to low-frequency oscillations when the data window is very short.
* Explore what happens to high-frequency oscillations when the sampling rate is low.


In [ ]:
# ASSIGMNENT:
# hint: modify the following code
ieeg, fig = sum_of_cosine_waves(
    duration = 10,
    fsample=500,
    cosine_wave_params = [
        {'amp':1,'freq':1,'phase':0},
        {'amp':1,'freq':20,'phase':0}
    ])

### Types of artifacts

Create an artificial signal with different types of artifacts:
* 10 s of data
* 1000 Hz sampling rate
* Pink noise as "true" neuronal background activity
* Waxing and waning 10 Hz oscillation as true brain signal
* Artifacts:
    * stationnary 50 Hz line noise
    * stationnary 20 Hz noise
    * 80 Hz noise burst 
    * signal jump
    * signal drift


In [ ]:
# Create signal
ieeg = {}
ieeg['srate'] = 1000
ieeg['dur']  = 10
ieeg['time'] = np.arange(0,ieeg['dur']*ieeg['srate'])/ieeg['srate']

ieeg['data'] = []
ieeg['type'] = []

# True signal: pink noise
ieeg['type'].append('EEG background data')
ieeg['data'].append(acoustics.generator.noise(
    N=len(ieeg['time']),        # 10`000 samples
    color='pink',  
    state=np.random.RandomState(seed=2022) # seed for random numbers generator
    ))

# true signal waxing and waning 10 Hz oscillatoin
ieeg['type'].append('Waxing & waning 10 Hz oscillations')
ieeg['data'].append((0.5+0.5*np.sin(ieeg['time']*0.2*2*np.pi))*np.sin(ieeg['time']*10*2*np.pi))

# add 50 Hz sine-wave
ieeg['type'].append('50 Hz noise')
ieeg['data'].append(np.sin(ieeg['time']*50*2*np.pi))


# add 20 Hz sine-wave
ieeg['type'].append('20 Hz noise')
ieeg['data'].append(np.sin(ieeg['time']*20*2*np.pi))

# add 80 Hz burst
ieeg['type'].append('80 Hz burst')
ieeg['data'].append(
    2*sc.stats.norm.pdf(ieeg['time'],loc=3,scale=.1)*np.sin(ieeg['time']*80*2*np.pi))

# add jump
ieeg['type'].append('signal jump')
ieeg['data'].append(
    np.zeros(ieeg['time'].shape)+5*np.ones(ieeg['time'].shape)*(ieeg['time']>7))


# add slow drift
ieeg['type'].append('signal drift')
ieeg['data'].append(4*np.sin((ieeg['time']+2)*0.05*2*np.pi))

# add sum of all signals
ieeg['type'].insert(0,'Sum of signals')
ieeg['data'].insert(0,np.row_stack(ieeg['data']).sum(axis=0))


In [ ]:
# Plotting functions

#%% Plot single signal
def plot_eeg_psd(ieeg,signal=0):
    
    fig, ax = plt.subplots(
        nrows=1,
        ncols=2)
    
    ieeg['psd_freqs']   = []
    ieeg['psd_pow']     = []
    
    
    # Compute PSD for all channels    
    
    tmp_analytic,tmp_freqs = mne.time_frequency.psd_array_multitaper(
        x = ieeg['data'][signal],
        sfreq = ieeg['srate'])
    
    ieeg['psd_freqs'].append(tmp_freqs)
    ieeg['psd_pow'].append(tmp_analytic)
    
    # Plot data
    ax[0].plot(ieeg['time'],ieeg['data'][signal],'r')
    ax[0].set_title(ieeg['type'][signal])
    ax[0].set_xlabel('Time (s)')
    ax[0].set_ylabel('Amplitude')
    ax[0].grid()
    
    ax[1].plot(ieeg['psd_freqs'][0],ieeg['psd_pow'][0],'r')                
    ax[1].set_xlabel('Frequency (Hz)')
    ax[1].set_ylabel('Amplitude')
    ax[1].set_yscale("log")
    ax[1].grid()
    ax[1].set_xlim(0,100)
    ax[1].set_yscale("log")
    return fig

def plot_multichannel_eeg_psd(ieeg):
    
    fig, ax = plt.subplots(
        nrows= len(ieeg['data']),
        ncols=2)
    
    ieeg['psd_freqs']   = []
    ieeg['psd_pow']     = []
    
    
    # Compute PSD for all channels    
    for SGi in range(len(ieeg['data'])):
        tmp_analytic,tmp_freqs = mne.time_frequency.psd_array_multitaper(
            x = ieeg['data'][SGi],
            sfreq = ieeg['srate'])
        
        ieeg['psd_freqs'].append(tmp_freqs)
        ieeg['psd_pow'].append(tmp_analytic)
    
    # Plot data
    for SGi in range(len(ieeg['data'])):
        ax[SGi,0].plot(ieeg['time'],ieeg['data'][0],'r')
        if SGi>0:
            ax[SGi,0].plot(ieeg['time'],ieeg['data'][SGi])
        ax[SGi,0].set_title(ieeg['type'][SGi])
        ax[SGi,0].set_xlabel('Time (s)')
        ax[SGi,0].set_ylabel('Amplitude')
        ax[SGi,0].grid()
        
        ax[SGi,1].plot(ieeg['psd_freqs'][0],ieeg['psd_pow'][0],'r')                
        if SGi>0:
            ax[SGi,1].plot(ieeg['psd_freqs'][SGi],ieeg['psd_pow'][SGi])
        ax[SGi,1].set_xlabel('Frequency (Hz)')
        ax[SGi,1].set_ylabel('Amplitude')
        ax[SGi,1].set_yscale("log")
        ax[SGi,1].grid()
        ax[SGi,1].set_xlim(0,100)
    return fig


Plot final signal with all "true" brain signals and artifacts:
* Red: final artificial signal (sum of all signals)
* Blue: specific signal components that are added to form the red signal

In [ ]:
fig = plot_multichannel_eeg_psd(ieeg)
fig.set_size_inches(15,20)
fig.set_dpi(60)
fig.tight_layout()

Explore single signals

In [ ]:
fig = plot_eeg_psd(ieeg,0)
fig.set_size_inches(15,5)
fig.set_dpi(60)
fig.tight_layout()

ASSIGNMENT: plot each artifact separately

In [ ]:
# ASSIGNMENT:
# => hint: use the above function to access the different signal in ieeg

### Artifact removal

#### Filtering
Types of filters:
* High-pass
* Low-pass
* Band-pass
* Notch

Additional information on filtering:
* de Cheveigné, A., & Nelken, I. (2019). Filters: When, Why, and How (Not) to Use Them. Neuron, 102(2), 280–293.
https://doi.org/10.1016/j.neuron.2019.02.039
* Filtering tutorial: https://mne.tools/stable/auto_tutorials/preprocessing/25_background_filtering.html



In [ ]:
#%% Keep only previously generated signal
ieeg_filt = copy.deepcopy(ieeg)

ieeg_filt['data']= [ieeg_filt['data'][0]]
ieeg_filt['type'] = ['Original']

Apply different types of filters

In [ ]:
# Apply different types of filters
ieeg_filt['type'].append('High-pass')
ieeg_filt['data'].append(
    mne.filter.filter_data(ieeg_filt['data'][0],
                           sfreq=ieeg_filt['srate'],
                           l_freq = 10,
                           h_freq = None))

ieeg_filt['type'].append('Low-pass')
ieeg_filt['data'].append(
    mne.filter.filter_data(ieeg_filt['data'][0],
                           sfreq=ieeg['srate'],
                           l_freq = None,
                           h_freq = 25))

ieeg_filt['type'].append('Band-pass')
ieeg_filt['data'].append(
    mne.filter.filter_data(ieeg_filt['data'][0],
                           sfreq=ieeg_filt['srate'],
                           l_freq = 10,
                           h_freq = 25))

ieeg_filt['type'].append('Notch')
ieeg_filt['data'].append(
    mne.filter.notch_filter(ieeg_filt['data'][0],
                            Fs=ieeg_filt['srate'],
                            freqs=[50]))


Plot the effect of distinct filters:
* Red: original signal
* Blue: signal after filtering


In [ ]:
fig = plot_multichannel_eeg_psd(ieeg_filt)
fig.set_size_inches(15,10)
fig.tight_layout()

ASSIGNMENT:
* Combine different filters to clean the signal 
* Plot the result

In [ ]:
# ASSIGNMENT
# => hint: adjust l_freq, h_freq, and freqs in following code
# Keep only the summed signal with artifacts
ieeg_clean = copy.deepcopy(ieeg)
ieeg_clean['data']= [ieeg_clean['data'][0]]
ieeg_clean['type'] = ['Cleaned']

# Filter the signal
ieeg_clean['data'][0] = mne.filter.filter_data(ieeg_clean['data'][0],
                           sfreq=ieeg_clean['srate'],
                           l_freq = 0,
                           h_freq = 200)


ieeg_clean['data'][0] = mne.filter.notch_filter(ieeg_clean['data'][0],
                            Fs=ieeg_clean['srate'],
                            freqs=[200])


fig = plot_eeg_psd(ieeg_clean,0)
fig.set_size_inches(15,5)

#### Filter artifacts
Filtering can introduce artifacts which may look like biologicas signals. E.g. ringing

Create and plot an artificial signal

In [ ]:
# Create a signal with a strong artifact => boxcar
artifact = {}

artifact['srate'] = 1000
artifact['dur']  = 10

artifact['time'] = np.arange(0,artifact['dur']*artifact['srate'])/artifact['srate']

artifact['data'] = [np.zeros(artifact['time'].shape)]
artifact['type'] = ['jump']

artifact_times = [4,6]
    
artifact['data'][0][((artifact['time']>min(artifact_times)) &
              (artifact['time'] < max(artifact_times)))] = artifact['data'][0][((artifact['time']>min(artifact_times)) &
                            (artifact['time'] < max(artifact_times)))]+1


In [ ]:
fig = plot_eeg_psd(artifact)
                                                                     
fig.set_size_inches(15,5)
fig.tight_layout()


Filter the signal and plot the result:
* Red: original signal
* Blue: filtered signal

In [ ]:
#%% Filter the signal to see/create artifacts
artifact_filt = copy.deepcopy(artifact)

artifact_filt['type'].append('High-pass')
artifact_filt['data'].append(
    mne.filter.filter_data(artifact_filt['data'][0],
                           sfreq=artifact_filt['srate'],
                           l_freq = 10,
                           h_freq = None))

artifact_filt['type'].append('Low-pass')
artifact_filt['data'].append(
    mne.filter.filter_data(artifact_filt['data'][0],
                           sfreq=artifact['srate'],
                           l_freq = None,
                           h_freq = 25))

artifact_filt['type'].append('Band-pass')
artifact_filt['data'].append(
    mne.filter.filter_data(artifact_filt['data'][0],
                           sfreq=artifact_filt['srate'],
                           l_freq = 10,
                           h_freq = 25))




fig = plot_multichannel_eeg_psd(artifact_filt)

fig.set_size_inches(15,10)
fig.tight_layout()


# PREPROCESSING: Practical example

Download intracranial eeg data from Fedele et al., 2017, Scientific Reports.
* Data repository: https://openneuro.org/datasets/ds003498/versions/1.0.1
* Original publication: *Fedele T, Burnos S, Boran E, Krayenbühl N, Hilfiker P, Grunwald T, Sarnthein J. Resection of high frequency oscillations predicts seizure outcome in the individual patient. Scientific Reports. 2017;7(1):13836. https://www.nature.com/articles/s41598-017-13064-1 doi:10.1038/s41598-017-13064-1*

Use MNE to process the data:
* https://mne.tools/stable/index.html

In [ ]:
import requests


# Dataset: events
url = "https://openneuro.org/crn/datasets/ds003498/snapshots/1.0.1/files/sub-01:ses-interictalsleep:ieeg:sub-01_ses-interictalsleep_run-01_events.tsv"
r = requests.get(url, allow_redirects=True)
open('sub-01_ses-interictalsleep_run-01_events.tsv', 'wb').write(r.content)


# Dataset: channel info
url = "https://openneuro.org/crn/datasets/ds003498/snapshots/1.0.1/files/sub-01:ses-interictalsleep:ieeg:sub-01_ses-interictalsleep_run-01_channels.tsv"
r = requests.get(url, allow_redirects=True)
open('sub-01_ses-interictalsleep_run-01_channels.tsv', 'wb').write(r.content)


# Dataset: header
url = "https://openneuro.org/crn/datasets/ds003498/snapshots/1.0.1/files/sub-01:ses-interictalsleep:ieeg:sub-01_ses-interictalsleep_run-01_ieeg.vhdr"
r = requests.get(url, allow_redirects=True)
open('sub-01_ses-interictalsleep_run-01_ieeg.vhdr', 'wb').write(r.content)

# Dataset: marker
url = "https://openneuro.org/crn/datasets/ds003498/snapshots/1.0.1/files/sub-01:ses-interictalsleep:ieeg:sub-01_ses-interictalsleep_run-01_ieeg.vmrk"
r = requests.get(url, allow_redirects=True)
open('sub-01_ses-interictalsleep_run-01_ieeg.vmrk', 'wb').write(r.content)

# Dataset: eeg
url = "https://openneuro.org/crn/datasets/ds003498/snapshots/1.0.1/files/sub-01:ses-interictalsleep:ieeg:sub-01_ses-interictalsleep_run-01_ieeg.eeg"
r = requests.get(url, allow_redirects=True)
open('sub-01_ses-interictalsleep_run-01_ieeg.eeg', 'wb').write(r.content)



Import the data using mne:

In [ ]:
ieeg = mne.io.read_raw_brainvision('sub-01_ses-interictalsleep_run-01_ieeg.vhdr',preload=True)
ieeg.annotations.crop(0,0)
ieeg.info

## Visual inspection

Plot first 10 s of raw data of all channels

In [ ]:
# set browser backend to matplotlib => no popup
mne.viz.set_browser_backend('matplotlib')

In [ ]:
# Plot data
fig = mne.viz.plot_raw(
    ieeg,
    duration=10.0,
    start=0.0,
    n_channels=50,
    show=False,
    scalings={'eeg':0.0001})

fig.set_size_inches(15,10)

Keep only data from macroelectrode targeting the left amydgala (AL1-8)

In [ ]:
ieeg_AL = ieeg.copy().pick_channels(['AL1','AL2','AL3','AL4','AL5','AL6','AL7','AL8'])

Plot raw data from left amydgala

In [ ]:
# Plot data
fig = mne.viz.plot_raw(
    ieeg_AL,
    duration=10.0,
    start=0.0,
    n_channels=50,
    show=False,
    scalings={'eeg':0.0001})

fig.set_size_inches(15,10)

ASSIGNMENT: explore other segments of the data

In [ ]:
# ASSIGNMENT
# => hint: use the above code, but change the start and duration parameters

Plot PSD of raw data:
* Use welch method:
    * Segment data into 2s windows that overlap by 90%
    * Compute PSD for each segment, and average
    
    

In [ ]:
fig = ieeg_AL.plot_psd(
    n_fft = int(2*ieeg_AL.info.get('sfreq')),          # => length of segments: 2* sampling frequency
    n_overlap = int(0.9*2*ieeg_AL.info.get('sfreq')),   # => overlap of segments: 0.9 (90%) of 2s windows
    show=False
)
fig.set_size_inches(15,5)

Visual inspection of the raw data reveals that:
* Channel 8 has no biological signal, is flat => drop
* All other channels are highly correlated, and that all channels have similar noise peaks in the PSD => shared noise component => re-reference data

Drop channel "AL8":

In [ ]:
ieeg_AL.drop_channels('AL8')

Re-reference the EEG to the common average of all channels:
* Compute the mean of all channels at each sample
* Subtract this mean from each single channel at each sampel
* => this removes the shared information from all channels (shared information: most likely noise)

In [ ]:
ieeg_AL.set_eeg_reference(ref_channels='average')

Plot re-referenced data from left amydgala

In [ ]:
# Plot data
fig = mne.viz.plot_raw(
    ieeg_AL,
    duration=10.0,
    start=0.0,
    n_channels=50,
    show=False,
    scalings={'eeg':0.0001})
fig.set_size_inches(15,10)

Plot PSD for rereferenced data from left amygdala

In [ ]:
fig = ieeg_AL.plot_psd(
    n_fft = int(2*ieeg_AL.info.get('sfreq')),          # => length of segments: 2* sampling frequency
    n_overlap = int(0.9*2*ieeg_AL.info.get('sfreq')),   # => overlap of segments: 0.9 (90%) of 2s windows
    show=False
)
fig.set_size_inches(15,5)

Focus PSD on frequncies < 200 Hz

In [ ]:
fig = ieeg_AL.plot_psd(
    n_fft = int(2*ieeg_AL.info.get('sfreq')),          # => length of segments: 2* sampling frequency
    n_overlap = int(0.9*2*ieeg_AL.info.get('sfreq')),  # => overlap of segments: 0.9 (90%) of 2s windows
    fmax = 200,                                         # => show data < 200 Hz
    show = False
)
fig.set_size_inches(15,5)

Visual inspection of re-referenced data reveals: 
* Most artifacts that were visible in the PSD are gone
* All channels show similar spectral compositions

## Channel statistics

Compute and report simple descriptive statistics for all channels:
* Mean of signal in time (should be == 0)
* Standard deviation of signal in time (should be similar across channels)
* Maximum absolute amplitude in each channel (should be similar across channel)
* Same statistics for 1st derivative ("diff") of signal (the change in amplituds between sample)
    * ...
* Normalize values by standard deviation to highlight outlier-channel


Observation:
* all channels have similar statistics (all statistics within +/- 2 stds) => no clear outlier

In [ ]:
# Compute statistics for each channel, make data frame
channel_stats = pd.DataFrame({
    'channel': ieeg_AL.ch_names,
    'signal_mean':np.mean(ieeg_AL.get_data(),axis=1),
    'signal_std': np.std(ieeg_AL.get_data(),axis=1),
    'signal_max': np.max(np.abs(ieeg_AL.get_data()),axis=1),
    'signaldiff_mean': np.mean(np.diff(ieeg_AL.get_data(),axis=1),axis=1),
    'signaldiff_std': np.std(np.diff(ieeg_AL.get_data(),axis=1),axis=1),
    'signaldiff_max': np.max(np.abs(np.diff(ieeg_AL.get_data(),axis=1)),axis=1),
})


# Normalize
channel_stats_norm = channel_stats.drop(columns=['channel'])
channel_stats_norm = (channel_stats_norm-channel_stats_norm.mean())/channel_stats_norm.std()

channel_stats_norm['channel'] = channel_stats['channel']

channel_stats_norm.set_index('channel',inplace=True)


channel_stats_norm

In [ ]:
fig, ax = plt.subplots(figsize= (15,10))
channel_stats_norm.plot(subplots = True,
                        ax = ax,
                        grid = True,
                        ylim=[-2, 2])


# FEATURE EXTRACTION
Once the signal is cleaned, we can extract specific features that can be used for further analyses (e.g. machine learning, or statistical testing)

Possible relevant features (to be discussed in the lecture):
* Power spectral density of a channel, segment, ...
* Time-frequency analyses
* Instantaneous phase and power:
  * => Phase-amplitude coupling
  * => Phase coherence
* Measures of coherence/connectivity (in the time-domain, in the spectral domain)
  * => network analyses
* Measures of signal complexity
* Distinct events:
  * High-frequency oscillation bursts
  * Single neuornal spikes
* ...  

Here, we can only touch a few features.

## Power spectral density (PSD)
* A time-series can be described as a sum of oscillations of different frequencies that all have different amplitudes
* Inspecting the different frequency contents (the amplitudes different frequencies have) can provide information about the source from which the signal is recorded.
 
For examples, see above

## Time-frequency analysis (TFA)

Definition: in time-frequency analyses, the signal in the time domain is transformed into the frequency domain for short subsegements of the original signal. This allows exploring how the spectral composition of the signal varies across time.
 
Basic principle:
* Cut the raw data into short (overlapping) segments
* Transform each segment to the frequency domain => PSD
* Assess how the spectral composition varies over time

### Example

Create an artificial signal:
* Sampling rate: 1000 Hz
* Duration: 5s
* Background eeg activity: pink noise
* 10 Hz sine wave burst from 2-3 seconds
* Continuous 20 Hz oscillation
* Chirp starting at 20 Hz with linare increase in frequency

In [ ]:
#%% Create artificial data
ieeg = {}
ieeg['srate'] = 1000
ieeg['dur']  = 5
ieeg['time'] = np.arange(0,ieeg['dur']*ieeg['srate'])/ieeg['srate']

ieeg['data'] = []
ieeg['type'] = []

# add some pink-noise
ieeg['type'].append('ieeg')
ieeg['data'].append(acoustics.generator.noise(
    N=len(ieeg['time']),        # 10`000 samples
    color='pink',  
    state=np.random.RandomState(seed=2022) # seed for random numbers generator
    ))

# add 10 Hz sine wave burst
ieeg['data'][0][
    (ieeg['time']>2) & (ieeg['time']<3)] = \
    ieeg['data'][0][(ieeg['time']>2) & (ieeg['time']<3)] +\
    5*np.sin(ieeg['time'][(ieeg['time']>2) & (ieeg['time']<3)]*2*np.pi*10)

# add 20 Hz fixed wave
ieeg['data'][0] = ieeg['data'][0] + 2*np.sin(ieeg['time']*2*np.pi*20)


# add a chirp that linearly increases with frequency
ieeg['data'][0] = ieeg['data'][0] + 2*np.sin(ieeg['time']*2*np.pi*(20+2*ieeg['time']))


Plot the artificial signal

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(ieeg['time'],
       ieeg['data'][0])


ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()

Perform the time-frequency decomposition:
* Perform Morlet wavelet decomposition
* Extract power for frequencies from 1 to 50 Hz at 1 Hz steps
* Number of wavelet cycles per frequency: Frequency/2 (e.g. 0.5 cyles at 1 Hz, 25 cycles at 50 Hz)

In [ ]:
# Decompose the signal
ieeg['tfr_freqs'] = np.arange(1,50)         # Frequency resolution of the decomposition
ieeg['tfr_ncycles'] = ieeg['tfr_freqs']/2   # Number of cycles (length of window) for which to compute the power at each frequency

ieeg['tfr'] = mne.time_frequency.tfr_array_morlet(
    epoch_data = ieeg['data'][0][np.newaxis,np.newaxis,:],
    sfreq = ieeg['srate'],
    freqs= ieeg['tfr_freqs'],
    n_cycles=ieeg['tfr_ncycles'])

Plot the decomposition
* all signal components become visible: 20 Hz continuous activity, 10 Hz burst, chirp with increasing frequency from 20 to 40 Hz

In [ ]:
# Plot the decomosition
fig, ax = plt.subplots(figsize= (15,5))

img = ax.pcolormesh(
    ieeg['time'],
    ieeg['tfr_freqs'],
    np.abs(np.squeeze(ieeg['tfr'])))

ax.set_ylabel('Frequency (Hz)')
ax.set_xlabel('Time (s)')

cb = plt.colorbar(img, ax=ax)
cb.set_label('Power')

ASSIGNMENT:
* Explore what happens if the frequency resolution is increased/decreased
* Explore what happens if the number of cycles is alters (e.g. fix numer of cycles for all frequencies)

In [ ]:
# ASSIGNMENT:
# => hint: modify the following elements of the above code:
#   ieeg['tfr_freqs'] = ...
#   ieeg['tfr_ncycles'] = ...

### Example

Perform the time-frequency decomposition on real data from Amydgala (channel AL1):
* Perform Morlet wavelet decomposition
* Extract power for frequencies from 1 to 50 Hz at 1 Hz steps
* Number of wavelet cycles per frequency: Frequency/2 (e.g. 0.5 cyles at 1 Hz, 25 cycles at 50 Hz)
* Normalize the signal: subtract mean in each frequency, divide by standard deviation

In [ ]:
# Decompose the signal
AL_dat = {}
AL_dat['tfr_freqs'] = np.arange(1,50)         # Frequency resolution of the decomposition
AL_dat['tfr_ncycles'] = AL_dat['tfr_freqs']/2   # Number of cycles (length of window) for which to compute the power at each frequency

AL_dat['tfr'] = mne.time_frequency.tfr_array_morlet(
    epoch_data = ieeg_AL.get_data('AL1')[np.newaxis,:],
    sfreq = ieeg_AL.info.get('sfreq'),
    freqs= AL_dat['tfr_freqs'],
    n_cycles=AL_dat['tfr_ncycles'])

In [ ]:
AL_dat['tfr'] = np.squeeze(AL_dat['tfr'])
AL_dat['tfr'] = (AL_dat['tfr']-np.mean(AL_dat['tfr'],axis=1)[:,None])/np.std(AL_dat['tfr'],axis=1)[:,None]

Plot the decomposition

In [ ]:
# Plot the decomposition
fig, ax = plt.subplots( figsize= (15,5))

img = ax.pcolormesh(
    ieeg_AL.times,
    AL_dat['tfr_freqs'],
    np.abs(np.squeeze(AL_dat['tfr'])))

ax.set_ylabel('Frequency (Hz)')
ax.set_xlabel('Time (s)')
ax.set_xlim(0,30)

cb = plt.colorbar(img, ax=ax)
cb.set_label('Power')

## Instantaneous power, phase, (frequency)

Definition: Analysis of the time-resolved (i.e., instantaneous) power and phase of a signal in a specific frequency band. 
 
Possible aplications:
* Assess power fluctuations in a specific frequency range => e.g. is how does beta (20 Hz) power in the motor cortex change during a motor task?
* Assess the specific phase of the signal at a given time => e.g. 
* Assess coherence / connectivity between different channels and or frequencies:
    * e.g. does activity (phase) in the delta band (< 4 Hz) modulate activity in the sigma band (12-16 Hz)?
    * does the connectivity between two brain signals increase? assess signal coherence in the beta range (coherence between beta phase values, correlation between amplitudes, ...)?

### Example

Create an artificial signal:
* Sampling rate: 1000 Hz
* Duration: 5s
* Background eeg activity: pink noise
* 10 Hz sine wave burst from 2-3 seconds
* Continuous 2 Hz oscillation
* 20 Hz oscillatoin that is modulated by 2 Hz oscillation



In [ ]:
#%% Create artificial data
ieeg = {}
ieeg['srate'] = 1000
ieeg['dur']  = 5
ieeg['time'] = np.arange(0,ieeg['dur']*ieeg['srate'])/ieeg['srate']

ieeg['data'] = []
ieeg['type'] = []

# add some pink-noise
ieeg['type'].append('ieeg')
ieeg['data'].append(acoustics.generator.noise(
    N=len(ieeg['time']),        # 10`000 samples
    color='pink',  
    state=np.random.RandomState(seed=2022) # seed for random numbers generator
    ))

# add 10 Hz sine wave burst
ieeg['data'][0][
    (ieeg['time']>2) & (ieeg['time']<3)] = \
    ieeg['data'][0][(ieeg['time']>2) & (ieeg['time']<3)] +\
    5*np.sin(ieeg['time'][(ieeg['time']>2) & (ieeg['time']<3)]*2*np.pi*10)

# add 4 Hz fixed wave
ieeg['data'][0] = ieeg['data'][0] + 2*np.sin(ieeg['time']*2*np.pi*2)


# add a chirp that linearly increases with frequency
ieeg['data'][0] = ieeg['data'][0] + 2*np.sin(ieeg['time']*2*np.pi*20)*(1+np.sin(ieeg['time']*2*np.pi*2))


Plot the artificial signal

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(ieeg['time'],
       ieeg['data'][0])


ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()

Perform the time-frequency decomposition:
* Perform Morlet wavelet decomposition
* Number of wavelet cycles per frequency: Frequency/2 
* Extract power for frequencies from 1 to 50 Hz at 1 Hz steps

In [ ]:
# Decompose the signal
ieeg['tfr_freqs'] = np.arange(1,50)         # Frequency resolution of the decomposition
ieeg['tfr_ncycles'] = ieeg['tfr_freqs']/2   # Number of cycles (length of window) for which to compute the power at each frequency

ieeg['tfr'] = mne.time_frequency.tfr_array_morlet(
    epoch_data = ieeg['data'][0][np.newaxis,np.newaxis,:],
    sfreq = ieeg['srate'],
    freqs= ieeg['tfr_freqs'],
    n_cycles=ieeg['tfr_ncycles'])

Plot the decomposition

In [ ]:
# Plot the decomosition
fig, ax = plt.subplots(figsize= (15,5))

img = ax.pcolormesh(
    ieeg['time'],
    ieeg['tfr_freqs'],
    np.abs(np.squeeze(ieeg['tfr'])))

ax.set_ylabel('Frequency (Hz)')
ax.set_xlabel('Time (s)')

cb = plt.colorbar(img, ax=ax)
cb.set_label('Power')

Pefrorm the hilbert transform for 20 Hz oscillations:
* Bandpass filter between 18-22 Hz
* Get the hilbert transform

In [ ]:
#%% Hilbert decomposition
sgn_raw     = ieeg['data'][0]
sgn_filt    = mne.filter.filter_data(sgn_raw,
                           sfreq=ieeg['srate'],
                           l_freq = 18,
                           h_freq = 22)

sgn_hilb20   = sc.signal.hilbert(x=sgn_filt)

Plot the hilbert transform

In [ ]:
fig, ax = plt.subplots(nrows = 2,figsize=(15,10))

ax[0].plot(ieeg['time'],sgn_raw, label='raw')
ax[0].plot(ieeg['time'],sgn_filt,label='bandpass filtered')
ax[0].plot(ieeg['time'],np.abs(sgn_hilb20),label='hilbert envelope',color='g')
ax[0].legend()
ax[0].grid()


ax[1].set_ylabel('Amplitude')
ax[1].plot(ieeg['time'],np.angle(sgn_hilb20),color='g')
ax[1].set_ylabel('Phase')
ax[1].grid()

Pefrorm the hilbert transform for 2 Hz oscillations:
* Bandpass filter between 1-3 Hz
* Get the hilbert transform

In [ ]:
#%% Hilbert decomposition
sgn_raw     = ieeg['data'][0]
sgn_filt    = mne.filter.filter_data(sgn_raw,
                           sfreq=ieeg['srate'],
                           l_freq = 1,
                           h_freq = 3)

sgn_hilb2   = sc.signal.hilbert(x=sgn_filt)

Plot the hilbert transform

In [ ]:
fig, ax = plt.subplots(nrows = 2,figsize=(15,10))

ax[0].plot(ieeg['time'],sgn_raw, label='raw')
ax[0].plot(ieeg['time'],sgn_filt,label='bandpass filtered')
ax[0].plot(ieeg['time'],np.abs(sgn_hilb2),label='hilbert envelope',color='g')
ax[0].legend()
ax[0].grid()


ax[1].set_ylabel('Amplitude')
ax[1].plot(ieeg['time'],np.angle(sgn_hilb2),color='g')
ax[1].set_ylabel('Phase')
ax[1].grid()

## Phase-amplitude coupling
Phase amplitude-coupling describes the phenomenon that the activity (amplitude) of a higher frequency (e.g. 20 Hz) is coupled to a specific phase (e.g. the peak) of a lower freqeuency oscillation (e.g. 2 Hz). The phase of a low frequency oscillation thus appears to modulate the amplitude of a high-frequency oscillation.

Relevance (see Munia & Aviyente, 2019;  https://doi.org/10.1038/s41598-019-48870-2):
* *"Various forms of neural synchrony between oscillations across different frequency bands have been suggested as the major mechanism of neural integration."* 
* *"Previous studies based on electrophysiological measurement of neural activity suggest that different frequency bands are responsible for distinct computational roles as oscillations are thought to create synchronization across specialized brain regions to corroborate cognitive processing."*
* *"The power and/or the synchronization measured across different frequency bands have been related to various cognitive and neuronal functions."*


Further reading:
* Different measures of PAC: *Combrisson, E., Nest, T., Brovelli, A., Ince, R. A. A., Soto, J. L. P., Guillot, A., & Jerbi, K. (2020). Tensorpac: An open-source Python toolbox for tensor-based phase-amplitude coupling measurement in electrophysiological brain signals. PLOS Computational Biology, 16(10), e1008302. https://doi.org/10.1371/journal.pcbi.1008302*
* On the relevance of PAC: *Munia, T. T. K., & Aviyente, S. (2019). Time-frequency based phase-amplitude coupling measure for neuronal oscillations. Scientific Reports, 9(1), 12441. https://doi.org/10.1038/s41598-019-48870-2* 

### Example

Create an artificial signal:
* Sampling rate: 1000 Hz
* Duration: 5s
* Background eeg activity: pink noise
* 10 Hz sine wave burst from 2-3 seconds
* Continuous 2 Hz oscillation
* 20 Hz oscillation that is modulated by 2 Hz oscillation => coupling between 2 Hz phase and 20 Hz amplitude


In [ ]:
#%% Create artificial data
ieeg = {}
ieeg['srate'] = 1000
ieeg['dur']  = 5
ieeg['time'] = np.arange(0,ieeg['dur']*ieeg['srate'])/ieeg['srate']

ieeg['data'] = []
ieeg['type'] = []

# add some pink-noise
ieeg['type'].append('ieeg')
ieeg['data'].append(acoustics.generator.noise(
    N=len(ieeg['time']),        # 10`000 samples
    color='pink',  
    state=np.random.RandomState(seed=2022) # seed for random numbers generator
    ))

# add 10 Hz sine wave burst
ieeg['data'][0][
    (ieeg['time']>2) & (ieeg['time']<3)] = \
    ieeg['data'][0][(ieeg['time']>2) & (ieeg['time']<3)] +\
    5*np.sin(ieeg['time'][(ieeg['time']>2) & (ieeg['time']<3)]*2*np.pi*10)

# add 2 Hz fixed wave
ieeg['data'][0] = ieeg['data'][0] + 2*np.sin(ieeg['time']*2*np.pi*2)


# Add a 20 Hz oscillation that is modulated by the phase of the 2 Hz oscillation
ieeg['data'][0] = (ieeg['data'][0] +                    # current data
                   2*np.sin(ieeg['time']*2*np.pi*20)*   # 20 Hz sine wave
                   (1+np.sin(ieeg['time']*2*np.pi*2)))  # Multiplied by amplitude of 2 Hz sine wave


Plot the artificial signal

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(ieeg['time'],
       ieeg['data'][0])


ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()

Perform the hilbert transform for the 20 Hz, 2 Hz, and 10 Hz band

In [ ]:
#%% Hilbert decomposition
sgn_raw     = ieeg['data'][0]

sgn_filt20    = mne.filter.filter_data(sgn_raw,
                           sfreq=ieeg['srate'],
                           l_freq = 18,
                           h_freq = 22)

sgn_hilb20   = sc.signal.hilbert(x=sgn_filt20)

sgn_filt2    = mne.filter.filter_data(sgn_raw,
                           sfreq=ieeg['srate'],
                           l_freq = 1,
                           h_freq = 3)

sgn_hilb2   = sc.signal.hilbert(x=sgn_filt2)

sgn_filt10    = mne.filter.filter_data(sgn_raw,
                           sfreq=ieeg['srate'],
                           l_freq = 8,
                           h_freq = 12)

sgn_hilb10   = sc.signal.hilbert(x=sgn_filt10)


Plot the raw signal, the filtered signal, and the envelopes

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols= 1, figsize=(15,15))
ax[0].plot(ieeg['time'],sgn_raw, label='raw')
ax[0].plot(ieeg['time'],sgn_filt2,label='bandpass filtered')
ax[0].plot(ieeg['time'],np.abs(sgn_hilb2),label='hilbert envelope',color='g')
ax[0].legend()
ax[0].grid()
ax[0].set_title('2 Hz oscillation')

ax[1].plot(ieeg['time'],sgn_raw, label='raw')
ax[1].plot(ieeg['time'],sgn_filt10,label='bandpass filtered')
ax[1].plot(ieeg['time'],np.abs(sgn_hilb10),label='hilbert envelope',color='g')
ax[1].legend()
ax[1].grid()
ax[1].set_title('10 Hz oscillation')



ax[2].plot(ieeg['time'],sgn_raw, label='raw')
ax[2].plot(ieeg['time'],sgn_filt20,label='bandpass filtered')
ax[2].plot(ieeg['time'],np.abs(sgn_hilb20),label='hilbert envelope',color='g')
ax[2].legend()
ax[2].grid()
ax[2].set_title('20 Hz oscillation')

fig.tight_layout()

Assess how activity at 10 Hz and 20 Hz is coupled to the 2 Hz phase
* Bandpasss filter the signal in the respective frequency bans
* Extract the instantaneous 2 Hz phase angles and 10 Hz and 20 Hz amplitude for each samples
* Assess the coupling between the 2 Hz phase angles and the 10/20 Hz values

Make a data frame with the amplitude values at 10 Hz and 20 Hz and the phase angles at 2 Hz for each sample:

In [ ]:
phase_amp_data = pd.DataFrame(
    {
        'phase2Hz': np.angle(sgn_hilb2),
        'amp20Hz':np.abs(sgn_hilb20),
        'amp10Hz':np.abs(sgn_hilb10)}
)

Group the phase angles into 24 equally sized bins

In [ ]:
phase_amp_data['phase2Hz_bin'] = pd.cut(phase_amp_data['phase2Hz'],24)
phase_amp_data['phase2Hz_bincentres'] = phase_amp_data['phase2Hz_bin'].apply(lambda x: x.mid)
phase_amp_data

Compute the mean amplitudes at 10 Hz and 20 Hz for each 2 Hz bin. Normalize the amplitudes in each bin by the sum of of the amplitudes from all bins.

In [ ]:
phase_amp_data_aggr = phase_amp_data.groupby('phase2Hz_bincentres').mean().reset_index()
phase_amp_data_aggr['amp10Hz']= phase_amp_data_aggr['amp10Hz']/sum(phase_amp_data_aggr['amp10Hz'])
phase_amp_data_aggr['amp20Hz']= phase_amp_data_aggr['amp20Hz']/sum(phase_amp_data_aggr['amp20Hz'])
phase_amp_data_aggr



Quantify phase amplitude coupling:
* If the 2 Hz oscillation modulates the amplitude of the high-frequency oscillation, high-frequency amplitudes should be significantly higher in specific phase bins of the 2 Hz oscillation
* This bias towards a phase can be expressed as "MODULATION INDEX": the amplitude of the mean vector computed over all phases and their amplitudes. 

Compute the mean vector as complex number:

In [ ]:
MI_2hzX10Hz = np.mean(
    phase_amp_data_aggr['amp10Hz'].to_numpy()*np.exp(1j*phase_amp_data_aggr['phase2Hz_bincentres'].to_numpy()))

MI_2hzX20Hz = np.mean(
    phase_amp_data_aggr['amp20Hz'].to_numpy()*np.exp(1j*phase_amp_data_aggr['phase2Hz_bincentres'].to_numpy()))



Plot amplitudes in each phase on polar coordinate system, add mean vector

In [ ]:
fig = plt.figure(figsize=(15,10))

ax = fig.add_subplot(121, projection='polar')

ax.bar(x=phase_amp_data_aggr['phase2Hz_bincentres'],
       height=phase_amp_data_aggr['amp20Hz'],
      width=2*np.pi/24,
       linewidth=2, 
       edgecolor="white")
ax.plot([0,np.angle(MI_2hzX20Hz)],
        [0,np.abs(MI_2hzX20Hz)],'r',linewidth=2)
ax.set_title('Mean 20 Hz amplitude by 2 Hz phase')
ax.set_ylim(0,1/12)


ax = fig.add_subplot(122, projection='polar')
ax.bar(x=phase_amp_data_aggr['phase2Hz_bincentres'],
       height= phase_amp_data_aggr['amp10Hz'],
       width=2*np.pi/24,
       linewidth=2, 
       edgecolor="white")
ax.plot([0,np.angle(MI_2hzX10Hz)],
        [0,np.abs(MI_2hzX10Hz)],'r',linewidth=2)
ax.set_ylim(0,1/12)

ax.set_title('Mean 10 Hz amplitude by 2 Hz phase')

### Comodulogram: PAC over range of frequencies
The comodulogram illustrates phase-amplitude coupling between phases of an entire range of frequencies (e.g. 1-6 Hz) and amplitudes of an entire range of frequencies (e.g. 7-40 Hz)
 
The package "tensorpac" (https://etiennecmb.github.io/tensorpac/) provides efficient and robust methods for PAC analyses.



Compute comodulogram for artificial data:

In [ ]:
import tensorpac

# Define a Pac object
p = tensorpac.Pac(idpac=(6,0,0), f_pha=(1,6,0.5,0.1), f_amp=(6,40,1,0.5))
# Filter the data and extract pac
xpac = p.filterfit(1000, sgn_raw)

# Prpare figure
fig, ax = plt.subplots(figsize=(15,10))


# plot Phase-Amplitude Coupling :
ax = p.comodulogram(xpac.mean(-1), cmap='Spectral_r', plotas='contour', ncontours=20,
               title=r'2hz phase$\Leftrightarrow$20Hz amplitude coupling',
               fz_title=14, fz_labels=13)

p.show()

Compute comodulogram data from the Amygdala ('AL1'):
* coupling between 1 Hz phase and 20 Hz amplitudes
* coupling between 2.5 Hz phase and 25 Hz amplitudes

In [ ]:
# Define a Pac object
p = tensorpac.Pac(idpac=(6,0,0), f_pha=(1,10,1,1), f_amp=(11,100,1,1))
# Filter the data and extract pac
xpac = p.filterfit(ieeg_AL.info.get('sfreq'), ieeg_AL.get_data(0))

# Prpare figure
fig, ax = plt.subplots(figsize=(15,10))


# plot Phase-Amplitude Coupling :
ax = p.comodulogram(xpac.mean(-1), cmap='Spectral_r', plotas='contour', ncontours=20,
               title=r'<10hz phase$\Leftrightarrow$>20Hz amplitude coupling',
               fz_title=14, fz_labels=13)

p.show()